# Solución Tarea 2 pregunta 1 (a)

# Funciones auxiliares
Para hacer esta pregunta, es necesario tener implementado un test de primalidad. Y para este test, es necesario tener implementado el algoritmo extendido de euclides.

In [1]:
import random

def es_potencia(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False


def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    r, s, t = _extended_euclid_base(b, a)
    return r, t, s


def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t


def _is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if es_potencia(n):
        return False
    
    found_negative = False
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

Utilizando la función ``_is_probably_prime``, definimos una función que genera números primos con un número de bits dado como parámetro. Nótese que esta función recibe un parámetro ``number`` en el cual se indica cuántos números primos se debe retornar (la función no verifica que estós números sean distintos).

In [2]:
def _generate_primes(bit_number, number = 2):
    # Definimos el mayor y menor número con la cantidad de bits requerida
    upper = 2 ** bit_number - 1
    lower = 2 ** (bit_number - 1)
       
    primes = []
    while len(primes) < number:
        r = random.randint(lower, upper)
        if _is_probably_prime(r):           
            primes.append(r)

    return primes

A continuación usted debe implementar la clase ``Receiver`` que representa a quien recibe los mensajes. Esta clase debe permitir generar la clave pública y la clave secreta de un usuario, entregar la clave pública, y decriptar mensajes. Esta clase debe ser implementada suponiendo que si la clave pública es ``N``, entonces los mensajes (textos planos) son números entre ``0`` y ``N-1``.  

In [3]:
class Receiver:
    def __init__(self, bit_len):
        """
        Arguments:
            bit_len: A lower bound for the number of bits of the public key N
        """
        P, Q = _generate_primes(max(bit_len // 2 + 1, 3))
        while P == Q:
            P, Q = _generate_primes(max(bit_len // 2 + 1, 3))
        self.public_key = P * Q
        self.secret_key = (P -1) * (Q - 1)
   

    def get_public_key(self):
        """
        Returns:
          public_key: Public key 
        """
        return self.public_key

    
    def decrypt(self, ciphertext):
        """
        Arguments:
          ciphertext: The ciphertext to decrypt
        Returns:
          message: The original message
        """
        N = self.public_key
        phi_N = self.secret_key
        B = _extended_euclid(phi_N, N)[1] % N
        
        return (((pow(ciphertext, phi_N, N ** 2) - 1)//N) * B) % N 

Y debe implementar una clase ``Sender`` que representa a quien envía los mensajes. Para inicializar un objeto de esta clase se debe entregar como parámetro una clave pública con la que luego se debe poder encriptar mensajes. Nuevamente para implementar esta clase debe suponer que los mensaje son números, en particular los textos cifrados son números entre ``0`` y ``N**2 - 1`` si la clave pública es ``N``.

In [4]:
class Sender:
    def __init__(self, public_key):
        """
        Arguments:
          public_key: The public key that will be used to encrypt messages
        """
        self.public_key = public_key
    
    
    def encrypt(self, message):
        """
        Arguments:
          message: The plaintext message to encrypt
        Returns:
          ciphertext: The encrypted message
        """
        N = self.public_key
        r = random.randint(1, N-1)
        while _extended_euclid(r, N)[0] > 1:
            r = random.randint(1, N-1)
            
        return (pow(N + 1, message, N ** 2) * pow(r, N, N ** 2)) % (N ** 2)

Para probar que todo funciona bien, primero creamos un receiver con una clave pública de al menos 1024 bits, y verificamos que el largo sea el correcto.

In [5]:
receiver = Receiver(1024)
print(f"Clave pública: {receiver.get_public_key()}")
print(f"Largo de la clave pública: {len(bin(receiver.get_public_key()))-2}")

Clave pública: 271970571477896163527527922511805572577459107988707072895101066910053271104834813836192933743885833436654835733707204988930458026665696146921273496644315417496438826524446757018312006316114198450013322320734850275036797921408462858910812586997612005672781344245269637602239078870960093387836349979616737129257
Largo de la clave pública: 1025


En segundo lugar, creamos un sender con la clave pública del receiver. 

In [6]:
sender = Sender(receiver.get_public_key())

En tercer lugar, definimos un mensaje para encriptar y decriptar. 

In [7]:
message = 1234567890
print(message)

1234567890


Nótese que estamos seguros que ``message`` es un número entre ``0`` y ``N - 1`` ya que ``N`` es un número con al menos 1024 bits.

En cuarto lugar, encriptamos el mensaje tres veces con el objetivo de mostrar que la encriptación aleatorizada de este esquema criptográfico genera cifrados distintos (con una probabilidad muy alta). 

In [8]:
cipher_1 = sender.encrypt(message)
cipher_2 = sender.encrypt(message)
cipher_3 = sender.encrypt(message)
print(f"Primer cifrado del mensaje: {cipher_1}")
print(f"Segundo cifrado del mensaje: {cipher_2}")
print(f"Tercer cifrado del mensaje: {cipher_3}")

Primer cifrado del mensaje: 57945856874569973031589175378187778004505012755906314637632565633312961981197278223548608926756963394037591702853747997743191686292278413765311589019007580829328028984411987176509161861233845756707715161581290362674499699189402961721559929450823984002544826096314709871087611932643939780022801141140151365984485429665138208489470642005649963337501256467660893156962264806959461638205849086915850520350129924314961298678727843576634830683604168108542817858274801424121987984478805207692911826646483127404774945117310733666515323341156516493903951667209348582766139891807473530049967537428823174306994583150569596385523
Segundo cifrado del mensaje: 4960415500681691901976851634670084880570132427542357730202422069878526653585715403829239229533507529025675605677228282822225322645038729986195309295627201249245202577239487027175684328551419571439027237535391408538513390403889025150452562523743257334053192955179218413324592804392061421227015139646046884116966495622636824182

Finalmente, decriptamos los tres cifrados esperando obtener el mensaje original ``1234567890``. 

In [9]:
plaintext_1 = receiver.decrypt(cipher_1)
plaintext_2 = receiver.decrypt(cipher_2)
plaintext_3 = receiver.decrypt(cipher_3)
print(f"Primer texto decriptado: {plaintext_1}")
print(f"Segundo texto decriptado: {plaintext_2}")
print(f"Tercer texto decriptado: {plaintext_3}")

Primer texto decriptado: 1234567890
Segundo texto decriptado: 1234567890
Tercer texto decriptado: 1234567890


Si hasta aquí funcionó todo bien, seguramente va a tener todo el puntaje en esta pregunta de la tarea :-)